# ML API Specification for EO Data Cubes
### Brian Pondi and Rolf Simoes

## Setup and Connection

First, we load the required libraries and connect to the OpenEO backend.
Install the `openeo` package variant that's on this GitHub repository: https://github.com/PondiB/openeo-python-client

In [ ]:
import openeo # type: ignore

In [ ]:
connection = openeo.connect(url="http://127.0.0.1:8000")

In [ ]:
connection.authenticate_basic("brian", "123456")

In [ ]:
connection.list_collection_ids()

connection.describe_collection("mpc-sentinel-2-l2a")

In [ ]:
connection.describe_collection("aws-sentinel-2-l2a")

In [ ]:
connection.describe_collection("aws-sentinel-2-l2a")

## Explore Available Processes

Let's examine the available processes on the backend, particularly focusing on ML-related ones.


In [ ]:
process_ids = [process["id"] for process in connection.list_processes()]
print("Available processes on this backend:")
for process_id in process_ids:
    print(f"- {process_id}")

## Examine ML Process Details

Let's look at the details of some ML processes to understand their parameters and requirements.


In [ ]:
connection.describe_process("mlm_class_random_forest")

In [ ]:
connection.describe_process("mlm_class_tempcnn")

## Load and Prepare Data

We'll load Sentinel-2 data and prepare it for our analysis.

In [ ]:
datacube = connection.load_collection(
    collection_id="aws-sentinel-2-l2a",
    spatial_extent={"west": -63.6078, "south": -8.95630, "east": -63.25790, "north": -8.72290},
    temporal_extent=["2022-01-01", "2022-12-31"],
    bands=["b02", "b03", "b04", "b08", "cloud"]
)

In [ ]:
datacube = datacube.process(
    process_id="cube_regularize",
    arguments={
        "data": datacube,
        "period": "P16D",  # 16 days or 1 month  regularization 
        "resolution": 300
    }
)

## Load Training Data

We'll load the pre-processed training data for deforestation in Rondonia.


In [ ]:
#rond_data = connection.readRDS("./data/samples_deforestation_rondonia_1M.rds")
rond_data = "https://github.com/e-sensing/sitsdata/raw/main/data/samples_deforestation_rondonia.rds"

## Initialize and Train the Model

Now we'll initialize the Temporal CNN model and train it with our data.

In [ ]:
tempcnn_model_init = connection.mlm_class_tempcnn(
    optimizer="adam",
    learning_rate=0.0005,
    seed=42,
    verbose=True
)

Fit the model using the training dataset

In [ ]:
tempcnn_model = tempcnn_model_init.fit(
    training_set= rond_data,
    target="label"
)

## Make Predictions

Apply the trained model to make predictions on new data.


In [ ]:
datacube =  tempcnn_model.predict(datacube)

## Save the Model

Save the trained model for future use.

In [ ]:
saved_model = tempcnn_model.save_ml_model(name ="tempcnn_rondonia_2022", tasks=["classification"]
                            , options={"accelerator":"macos-arm", "framework":"Torch for R"}
                            )

## Save and Execute Results

Finally, we'll save the prediction results and execute the job.


In [ ]:
result = datacube.save_result(
    format="GeoTiff"
)

In [ ]:
result

In [ ]:
job = result.create_job(
    title="Deforestation Prediction in Rondonia",
    description="Using TempCNN model to predict deforestation in Rondonia"
)

In [ ]:
job.start_and_wait()
results = job.get_results()

In [ ]:
results.download_files("data/outputs/")

## Conclusion

This example demonstrated how to:
1. Connect to an OpenEO backend
2. Load and prepare training data
3. Define a Temporal CNN model architecture
4. Train the model
5. Make predictions on new data
6. Save the results

The trained model can now be used for making predictions on new time series data. 